In [65]:
import sys
sys.path.append("..")

from citibikeAPI import *
from utils import *


In [87]:
d = getAllStationData()
trip_data = pd.read_csv('../../raw/202003-citibike-tripdata.csv')
adjacenyDict = findMostCommonRoutesByStation(trip_data)


In [67]:
def findMostCommonRoutesByStation(trip_data):
	station_data = getAllStationData()
	#we only want trips that dont start and end at the same station
	trips = trip_data[trip_data['start station id'] != trip_data['end station id']].value_counts(['start station name','end station name'])
	#find unique starting stations
	unique_start_stations = list(set([trips.index[i][0] for i in range(len(trips))]))
	adjacencyDict = {}
	for station in unique_start_stations: 
		try: 
			id = station_data.index[station_data['name']==station][0]
			top_ten_routes = trips[station][:10]
			edges = []
			for route in top_ten_routes.index: 
				edges.append(station_data.index[station_data['name']==route][0])
			adjacencyDict[id] = edges
		#sometimes we get an index error if a station went online / offline since the last data pull 
		except IndexError: 
			pass 
	return adjacencyDict

In [89]:
d.head()
locData = d
locData = locData.reset_index()
locData.head()

,station_id,name,lat,lon,capacity,legacy_id,num_docks_available,num_bikes_disabled,num_ebikes_available,num_bikes_available,num_docks_disabled,station_status,is_renting,is_returning,last_reported,is_installed
0,41495491-5d89-4e14-aab9-c3db04aad399,43 St & Skillman Ave,40.746927,-73.920825,19,4774,16,0,0,2,0,active,1,1,1680612647,1
1,66dc686c-0aca-11e7-82f6-3863bb44ef7c,8 Ave & W 33 St,40.751551,-73.993934,96,490,90,1,1,4,0,active,1,1,1680612647,1
2,59b747c1-fbfb-4a3d-9394-8d06834d7c98,Sherman Ave & Thayer St,40.863124,-73.926994,21,4459,19,1,1,1,0,active,1,1,1680612647,1
3,74723006-b32f-4590-8cc7-f15eec3808fe,Bushwick Ave & Linden St,40.691460,-73.921460,29,3836,29,0,0,0,0,active,1,1,1680612648,1
4,66de85d2-0aca-11e7-82f6-3863bb44ef7c,Grand St & Elizabeth St,40.718822,-73.995960,45,3435,18,2,0,24,0,active,1,1,1680612648,1


In [98]:
#min max lat/long for normalize
# need to filter out zeros for some data noise 
minlat = min([i for i in d['lat'] if i > 0])
maxlat  = max([i for i in d['lat'] if i > 0])
minlong = min([i for i in d['lon'] if i  < 0])
maxlong = max([i for i in d['lon'] if i < 0])

#add two columns and normalize
#hardcoding column locations to overwrite ""
d['x'] = 0
d['y'] = 0
d['adjacency'] = ''
locData = d
locData = locData.reset_index()
print(adjacenyDict)
for r in range(d.shape[0]):
    d.iloc[r,-3] = clamp((d.iloc[r, 1] - minlat)/(maxlat-minlat),0,1) #lat
    d.iloc[r,-2] = clamp((d.iloc[r, 2] - minlong)/(maxlong-minlong),0,1) #lat
    try: 
        d.iloc[r, -1] = ' '.join([str(locData.index[locData['station_id']==v][0]) for v in adjacenyDict[locData.iloc[r,0]]])
    except KeyError: pass

locData = d[['x', 'y', 'adjacency']]


{'66db3273-0aca-11e7-82f6-3863bb44ef7c': ['b30815c0-99b6-451b-be15-902992cb8abb', '66db65aa-0aca-11e7-82f6-3863bb44ef7c', 'd9160982-2d9b-4f08-9469-a559a7b62809', '66de85d2-0aca-11e7-82f6-3863bb44ef7c', '66dbce8a-0aca-11e7-82f6-3863bb44ef7c', '66db6387-0aca-11e7-82f6-3863bb44ef7c', '66dc91b5-0aca-11e7-82f6-3863bb44ef7c', '66dbe4db-0aca-11e7-82f6-3863bb44ef7c', '66db2fd0-0aca-11e7-82f6-3863bb44ef7c', '66dc481a-0aca-11e7-82f6-3863bb44ef7c'], '66dde76a-0aca-11e7-82f6-3863bb44ef7c': ['66dde630-0aca-11e7-82f6-3863bb44ef7c', '66ddef96-0aca-11e7-82f6-3863bb44ef7c', '66de0dfd-0aca-11e7-82f6-3863bb44ef7c', '66ddea2a-0aca-11e7-82f6-3863bb44ef7c', '66de13dc-0aca-11e7-82f6-3863bb44ef7c', '66dde845-0aca-11e7-82f6-3863bb44ef7c', '66de2a00-0aca-11e7-82f6-3863bb44ef7c', '66de677e-0aca-11e7-82f6-3863bb44ef7c', '66dd448f-0aca-11e7-82f6-3863bb44ef7c', '66dde208-0aca-11e7-82f6-3863bb44ef7c'], '66dd1c7f-0aca-11e7-82f6-3863bb44ef7c': ['7a771842-c090-4da1-8c13-72b7f4f50a87', '1b28836b-29fd-47a7-9a98-44cd79291

In [99]:
locData.head()

,x,y,adjacency
station_id,,,
41495491-5d89-4e14-aab9-c3db04aad399,0.456220,0.808162,
66dc686c-0aca-11e7-82f6-3863bb44ef7c,0.474801,0.451968,
59b747c1-fbfb-4a3d-9394-8d06834d7c98,0.923110,0.778105,
74723006-b32f-4590-8cc7-f15eec3808fe,0.233350,0.805068,282 1157 653 1414 1036 664 879 1296 1367 1674
66de85d2-0aca-11e7-82f6-3863bb44ef7c,0.343293,0.442097,226 1042 1292 1762 826 781 1430 945 1053 1797


In [44]:
locData.to_csv('../../clean/locdata.csv')